In [5]:
import ipywidgets as widgets
from ipywidgets import GridBox, Layout, interactive_output
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint


# Names for the central partner and the two lovers
central_partner = "Kathe"
lover1 = "Jules"
lover2 = "Jim"

# Kathe's reaction function to love from Jules
def RL12(x21, tauI12, sigmaL12, sigmaI12, beta12):
    if x21 >= tauI12:
        return beta12 * x21 / (1 + x21/sigmaL12) * (1 - ((x21 - tauI12) / sigmaI12)**2) / (1 + ((x21 - tauI12) / sigmaI12)**2)
    else:
        return beta12 * x21 / (1 + x21/sigmaL12)

# Synergism function of Kathe (j=2,3)
def S(x1j, tau_S, sigmaS, s):
    if x1j >= tau_S:
        return s*((x1j - tau_S) / sigmaS)**2 / (1 + ((x1j - tau_S) / sigmaS)**2)
    else:
        return 0

# Platonicity function as defined by Jules
def P(x21, tauP, p, sigmaP):
    if x21 >= tauP:
        return p*((x21 - tauP) / sigmaP)**2 / (1 + ((x21 - tauP) / sigmaP)**2)
    else:
        return 0

# Jim's reaction function to love from Kathe
def RL31(x13, tauI31, beta31, sigmaL31, sigmaI31):
    if x13 >= tauI31:
        return beta31 * x13 / (1 + x13/sigmaL31) * (1 - ((x13 - tauI31) / sigmaI31)**2) / (1 + ((x13 - tauI31) / sigmaI31)**2)
    else:
        return beta31 * x13 / (1 + x13/sigmaL31)

def love_dynamics(y, t, params):
    x12, x13, x21, x31 = y
    alpha1, alpha2, alpha3, beta21, beta12, beta13, beta31, gamma1, gamma2, gamma3, epsilon, delta, A1, A2, A3, tauI12, sigmaL12, sigmaI12, tau_S, sigmaS, tauP, p, sigmaP, tauI31, sigmaL31, sigmaI31, s = params

    dx12dt = -alpha1 * np.exp(epsilon * (x13 - x12)) * x12 + RL12(x21, tauI12, sigmaL12, sigmaI12, beta12) + (1 + S(x12, tau_S, sigmaS, s)) * gamma1 * A2
    dx13dt = -alpha1 * np.exp(epsilon * (x12 - x13)) * x13 + beta13 * x31 + (1 + S(x13, tau_S, sigmaS, s)) * gamma1 * A3
    dx21dt = -alpha2 * x21 + beta21 * x12 * np.exp(delta * (x13 - x12)) + (1 - P(x21, tauP, p, sigmaP)) * gamma2 * A1
    dx31dt = -alpha3 * x31 + RL31(x13, tauI31, beta31, sigmaL31, sigmaI31) * np.exp(delta * (x13 - x12)) + gamma3 * A1

    return [dx12dt, dx13dt, dx21dt, dx31dt]

slider_style = {'description_width': 'initial'}
slider_layout = Layout(width='auto')

# Parameters
alpha1_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.01, description='Forgetting Coef. Kathe (alpha1)', style=slider_style, layout=slider_layout)
alpha2_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.01, description='Forgetting Coef. Jules (alpha2)', style=slider_style, layout=slider_layout)
alpha3_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.01, description='Forgetting Coef. Jim (alpha3)', style=slider_style, layout=slider_layout)
beta21_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.01, description='Reaction Coef. Jules to Kathe (beta21)', style=slider_style, layout=slider_layout)
beta12_slider = widgets.FloatSlider(value=8, min=0, max=20, step=0.01, description='Reaction Coef. Kathe to Jules (beta12)', style=slider_style, layout=slider_layout)
beta13_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.01, description='Reaction Coef. Kathe to Jim (beta13)', style=slider_style, layout=slider_layout)
beta31_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.01, description='Reaction Coef. Jim to Kathe (beta31)', style=slider_style, layout=slider_layout)
gamma1_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.01, description='Appeal Coef. Kathe (gamma1)', style=slider_style, layout=slider_layout)
gamma2_slider = widgets.FloatSlider(value=0.5, min=0, max=5, step=0.01, description='Appeal Coef. Jules (gamma2)', style=slider_style, layout=slider_layout)
gamma3_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.01, description='Appeal Coef. Jim (gamma3)', style=slider_style, layout=slider_layout)
epsilon_slider = widgets.FloatSlider(value=0.0062, min=0, max=0.05, step=0.0001, description='Sensitivity Coef. Kathe (epsilon)', style=slider_style, layout=slider_layout)
delta_slider = widgets.FloatSlider(value=0.0285, min=0, max=0.1, step=0.0001, description='Sensitivity Coef. Jules and Jim (delta)', style=slider_style, layout=slider_layout)
A1_slider = widgets.FloatSlider(value=20, min=0, max=100, step=0.1, description='Appeal of Kathe (A1)', style=slider_style, layout=slider_layout)
A2_slider = widgets.FloatSlider(value=4, min=0, max=50, step=0.1, description='Appeal of Jules (A2)', style=slider_style, layout=slider_layout)
A3_slider = widgets.FloatSlider(value=5, min=0, max=50, step=0.1, description='Appeal of Jim (A3)', style=slider_style, layout=slider_layout)
tauI12_slider = widgets.FloatSlider(value=2.5, min=0, max=20, step=0.1, description='Insecurity Threshold Kathe-Jules (tauI12)', style=slider_style, layout=slider_layout)
sigmaL12_slider = widgets.FloatSlider(value=10, min=0, max=50, step=0.1, description='Sensitivity of Reaction Kathe-Jules (sigmaL12)', style=slider_style, layout=slider_layout)
sigmaI12_slider = widgets.FloatSlider(value=10.5, min=0, max=50, step=0.1, description='Sensitivity of Insecurity Kathe-Jules (sigmaI12)', style=slider_style, layout=slider_layout)
tau_S_slider = widgets.FloatSlider(value=9, min=0, max=20, step=0.1, description='Synergism Threshold Kathe (tau_S)', style=slider_style, layout=slider_layout)
sigmaS_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='Sensitivity of Synergism Kathe (sigmaS)', style=slider_style, layout=slider_layout)
tauP_slider = widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='Platonicity Threshold Jules (tauP)', style=slider_style, layout=slider_layout)
p_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='Maximum Platonicity Jules (p)', style=slider_style, layout=slider_layout)
sigmaP_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='Sensitivity of Platonicity Jules (sigmaP)', style=slider_style, layout=slider_layout)
tauI31_slider = widgets.FloatSlider(value=9, min=0, max=20, step=0.1, description='Insecurity Threshold Jim-Kathe (tauI31)', style=slider_style, layout=slider_layout)
sigmaL31_slider = widgets.FloatSlider(value=10, min=0, max=50, step=0.1, description='Sensitivity of Reaction Jim-Kathe (sigmaL31)', style=slider_style, layout=slider_layout)
sigmaI31_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='Sensitivity of Insecurity Jim-Kathe (sigmaI31)', style=slider_style, layout=slider_layout)
s_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.1, description='Synergism Coefficient Kathe (s)', style=slider_style, layout=slider_layout)

# Arrange sliders in a grid
sliders = [alpha1_slider, alpha2_slider, alpha3_slider, 
           beta21_slider, beta12_slider, beta13_slider, beta31_slider, 
           gamma1_slider, gamma2_slider, gamma3_slider, 
           epsilon_slider, delta_slider, 
           A1_slider, A2_slider, A3_slider, tauI12_slider, 
           sigmaL12_slider, sigmaI12_slider, tau_S_slider, sigmaS_slider, tauP_slider, p_slider, sigmaP_slider, tauI31_slider, sigmaL31_slider, sigmaI31_slider, s_slider]

initial_conditions = [0, 0, 0, 0]
t = np.linspace(0, 20, 1000) 

def update_plot(alpha1, alpha2, alpha3, beta21, beta12, beta13, beta31, gamma1, gamma2, gamma3, epsilon, delta, A1, A2, A3, tauI12, sigmaL12, sigmaI12, tau_S, sigmaS, tauP, p, sigmaP, tauI31, sigmaL31, sigmaI31, s):
    params = [alpha1, alpha2, alpha3, beta21, beta12, beta13, beta31, gamma1, gamma2, gamma3, epsilon, delta, A1, A2, A3, tauI12, sigmaL12, sigmaI12, tau_S, sigmaS, tauP, p, sigmaP, tauI31, sigmaL31, sigmaI31, s]
    solution = odeint(love_dynamics, initial_conditions, t, args=(params,))
    x12, x13, x21, x31 = solution.T
    
    plt.clf()
    fig, axs = plt.subplots(3, 2, figsize=(12, 12))

    # Dynamics between central partner and lover1
    axs[0, 0].plot(t, x12, label=f'{central_partner} to {lover1}', color='tab:pink')
    axs[0, 0].plot(t, x21, label=f'{lover1} to {central_partner}', color='tab:blue')
    axs[0, 0].set_xlabel('Time (years)')
    axs[0, 0].set_ylabel('Feelings')
    axs[0, 0].set_title(f'{central_partner}-{lover1} Dynamics')
    axs[0, 0].legend()
    axs[0, 0].grid(True)

    # Phase diagram between central partner and lover1
    axs[0, 1].plot(x12, x21)
    axs[0, 1].set_xlabel(f'Feelings of {central_partner} towards {lover1}')
    axs[0, 1].set_ylabel(f'Feelings of {lover1} towards {central_partner}')
    axs[0, 1].set_title('Phase Diagram')
    axs[0, 1].grid(True)

    # Dynamics between central partner and lover2
    axs[1, 0].plot(t, x13, label=f'{central_partner} to {lover2}', color='tab:pink')
    axs[1, 0].plot(t, x31, label=f'{lover2} to {central_partner}', color='tab:green')
    axs[1, 0].set_xlabel('Time (years)')
    axs[1, 0].set_ylabel('Feelings')
    axs[1, 0].set_title(f'{central_partner}-{lover2} Dynamics')
    axs[1, 0].legend()
    axs[1, 0].grid(True)

    # Phase diagram between central partner and lover2
    axs[1, 1].plot(x13, x31)
    axs[1, 1].set_xlabel(f'Feelings of {central_partner} towards {lover2}')
    axs[1, 1].set_ylabel(f'Feelings of {lover2} towards {central_partner}')
    axs[1, 1].set_title('Phase Diagram')
    axs[1, 1].grid(True)

    # Imbalance plot
    imbalance = x12 - x13
    zero_crossings = np.where(np.diff(np.sign(imbalance)))[0]
    times_of_zero_crossings = t[zero_crossings] + \
                            (t[zero_crossings + 1] - t[zero_crossings]) * \
                            (0 - imbalance[zero_crossings]) / \
                            (imbalance[zero_crossings + 1] - imbalance[zero_crossings])

    axs[2, 0].plot(t, imbalance, color='tab:pink')
    axs[2, 0].scatter(times_of_zero_crossings, np.zeros_like(times_of_zero_crossings), color='red', zorder=5)
    axs[2, 0].fill_between(t, imbalance, where=(imbalance > 0), color='tab:blue', alpha=0.3)
    axs[2, 0].fill_between(t, imbalance, where=(imbalance < 0), color='tab:green', alpha=0.3)
    axs[2, 0].axhline(y=0, color='black', linestyle='--')
    axs[2, 0].set_xlabel('Time (years)')
    axs[2, 0].set_ylabel(f'Imbalance of {central_partner}\'s feelings \n towards {lover1} (>0) and {lover2} (<0)')
    axs[2, 0].set_title('Imbalance Over Time')
    axs[2, 0].grid(True)

    number_of_zeros = len(times_of_zero_crossings) - 2
    axs[2, 1].text(0.5, 0.5, f'Partner switching: {number_of_zeros}', 
                horizontalalignment='center', 
                verticalalignment='center', 
                fontsize=16, 
                transform=axs[2, 1].transAxes)
    axs[2, 1].axis('off')

    fig.tight_layout()
    plt.show()

slider_dict = {
    'alpha1': alpha1_slider, 
    'alpha2': alpha2_slider, 
    'alpha3': alpha3_slider, 
    'beta21': beta21_slider,
    'beta12': beta12_slider,
    'beta13': beta13_slider,
    'beta31': beta31_slider,
    'gamma1': gamma1_slider,
    'gamma2': gamma2_slider,
    'gamma3': gamma3_slider,
    'epsilon': epsilon_slider,
    'delta': delta_slider,
    'A1': A1_slider,
    'A2': A2_slider,
    'A3': A3_slider,
    'tauI12': tauI12_slider,
    'sigmaL12': sigmaL12_slider,
    'sigmaI12': sigmaI12_slider,
    'tau_S': tau_S_slider,
    'sigmaS': sigmaS_slider,
    'tauP': tauP_slider,
    'p': p_slider,
    'sigmaP': sigmaP_slider,
    'tauI31': tauI31_slider,
    'sigmaL31': sigmaL31_slider,
    'sigmaI31': sigmaI31_slider,
    's': s_slider
}

interactive_plot = interactive_output(update_plot, slider_dict)

display(GridBox(sliders, layout=Layout(
    width='100%',
    grid_template_columns='repeat(3, 32%)',  
    grid_gap='20px 20px'
)), interactive_plot)






GridBox(children=(FloatSlider(value=2.0, description='Forgetting Coef. Kathe (alpha1)', layout=Layout(width='a…

Output()